In [1]:
import os
import json
import numpy as np

In [2]:
from radismart.cardiac.perfusion.functions import gen_defect_extent_bull_eye_map, gen_defect_severity_bull_eye_map,gen_score_bull_eye
from radismart.cardiac.perfusion.functions import cal_defect_extent, cal_defect_severity,cal_tdp,cal_categoric_scores

In [3]:
cardiac_dir = os.path.dirname(os.path.dirname(os.getcwd()))
normal_limits_dir = os.path.join(cardiac_dir, 'resource', 'perfusion', 'normal_limits')

In [4]:
def load_segments(base_dir=os.path.join(cardiac_dir, 'resource', 'perfusion')):
    segment17 = np.load(os.path.join(base_dir, 'segment17.npy')).astype(np.float32)
    segment_coronary_artery = np.load(os.path.join(base_dir,  'segment_coronary_artery.npy')).astype(np.float32)
    segment_myocardial_wall = np.load(os.path.join(base_dir,  'segment_myocardial_wall.npy')).astype(np.float32)

    return segment17, segment_coronary_artery, segment_myocardial_wall


def calculate_segment_categoric_scores(patient_be, limits_type='male_rest'):
    normal_limits = np.load(os.path.join(normal_limits_dir,  f'{limits_type}_normal_limits.npy')).astype(np.float32)
    # normal_limits_sd = np.load(os.path.join(normal_limits_dir,  f'{limits_type}_sd.npy')).astype(np.float32)

    segment17, segment_coronary_artery, segment_myocardial_wall = load_segments()

    categoric_score_bull_eye = gen_score_bull_eye(patient_be, normal_limits)
    categoric_scores_seg17 = cal_categoric_scores(categoric_score_bull_eye, segment17)
    categoric_scores_cor = cal_categoric_scores(categoric_score_bull_eye, segment_coronary_artery)
    categoric_scores_myo = cal_categoric_scores(categoric_score_bull_eye, segment_myocardial_wall)
    return {
        "Segment17Score":categoric_scores_seg17,
        "SegmentConoraryScore":categoric_scores_cor,
        "SegmentMyocardiumScore":categoric_scores_myo
    }







In [5]:


map_dict={
    1.0: 'LCX', 
    2.0: 'RCA',
    3.0: 'LAD',
    'total': 'TOT'
}

def calculate_conorary_score(patient_be, limits_type='male_rest', info=map_dict):
    normal_limits = np.load(os.path.join(normal_limits_dir,  f'{limits_type}_normal_limits.npy')).astype(np.float32)
    normal_limits_sd = np.load(os.path.join(normal_limits_dir,  f'{limits_type}_sd.npy')).astype(np.float32)

    segment17, segment_coronary_artery, segment_myocardial_wall = load_segments()

    defect_extent_bull_eye = gen_defect_extent_bull_eye_map(patient_be, normal_limits)
    defect_extent_dict_cor = cal_defect_extent(defect_extent_bull_eye, segment_coronary_artery)

    score_bull_eye = gen_score_bull_eye(patient_be, normal_limits)
    tpd_dict_cor = cal_tdp(score_bull_eye, segment_coronary_artery)

    # print(defect_extent_dict_cor)
    res = {}
    for key, name in map_dict.items():
        res[name] = {
            'EXT': defect_extent_dict_cor.get(key),
            'TPD': tpd_dict_cor.get(key)
        }
    return res




import SimpleITK as sitk
resource_path = '../../resource'
os.path.exists(resource_path)

commercial_software_gen_cad_bull_eye = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(resource_path, 'perfusion', 'BullEye_cad'))).astype(np.float32)[1]
# normal_limits_bull_eye = np.load(os.path.join(resource_path, 'perfusion', 'normal_limits', 'male_rest_normal_limits.npy')).astype(np.float32)

calculate_conorary_score(commercial_software_gen_cad_bull_eye)






{'LCX': {'EXT': 12.23758320532514, 'TPD': 7.829227298498154},
 'RCA': {'EXT': 12.785947712418299, 'TPD': 8.442959934473038},
 'LAD': {'EXT': 16.049651567944252, 'TPD': 12.477333843708038},
 'TOT': {'EXT': 14.333370399199378, 'TPD': 10.369706153869629}}

In [6]:
def calculate_IschemiaResult(patient_be, limits_type='male_rest', info=map_dict):
    res = calculate_conorary_score(patient_be, limits_type, info)
    return {
        'Area': 105,
        'Defect': res['TOT']['EXT']*105/100,
        'Extent': res['TOT']['EXT'],
        'TPD': res['TOT']['TPD']
    }
    # return 

calculate_IschemiaResult(commercial_software_gen_cad_bull_eye)

{'Area': 105,
 'Defect': 15.050038919159347,
 'Extent': 14.333370399199378,
 'TPD': 10.369706153869629}